In [9]:
from genepy.utils import helper as h

from taigapy import TaigaClient
from celligner import Celligner
tc = TaigaClient()
from bokeh.plotting import output_notebook
from depmapomics import tracker as track
#autoreload
%load_ext autoreload
%autoreload 2
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [2]:
# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/
# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here:
TCGA_expression = tc.get(name='celligner-input-9827',
                         file='tumor_expression')

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [3]:
# subset gene names to ensembl ids only
CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

common = set(CCLE_expression.columns).intersection(
    set(TCGA_expression.columns))
CCLE_expression = CCLE_expression[list(common)]
TCGA_expression = TCGA_expression[list(common)]

In [4]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY/edit#gid=746987867 
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually, found also here: 

No dataset version provided. Using version 1.


In [5]:
# transforming annotations
CCLE_annotation = CCLE_annotation.drop_duplicates('arxspan_id').set_index("arxspan_id")
CCLE_annotation = CCLE_annotation.loc[CCLE_expression.index, ["origin", 'subtype']].rename(columns={"origin": "tissue_type", "subtype": 'disease_type'})
CCLE_annotation["cell_type"] = "cancer cell line"

TCGA_annotation = TCGA_annotation.set_index("sampleID").loc[TCGA_expression.index,["lineage",
"subtype"]].rename({"lineage":"tissue_type", "subtype": 'disease_type'})
TCGA_annotation['cell_type'] = "tumor sample"

In [10]:
my_alligner = Celligner(snn_kwargs={"neighbor_num": 30, "min_shared_neighbor_proportion": 1/15,}, make_plots=True)

my_alligner.fit(TCGA_expression, TCGA_annotation)

fetching gene names from biomart cache
using only usefull genes
looking at 12236 samples.
found 29593 common genes
creating a fit dataset..
reducing dimensionality...
clustering...


doing differential expression analysis on the clusters
running differential expression on 145 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
done


In [12]:
my_alligner.transform(CCLE_expression, CCLE_annotation)

/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:30: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 1d, A))
  dist[i, j] = np.dot(m[i], n[j])
/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:205: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, C), array(float32, 1d, A))
  scale = np.dot(working, grad)
/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:215: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, C), array(float32, 1d, A))
  curproj = np.dot(grad, curcell)
/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:219: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, C), array(float32, 1d, A))
  sameproj = np.dot(grad, samecell)
/home/jeremie/celligner/celligner/mnnpy/mnnpy/utils.py:229: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called 

looking at 2822 samples.
found 29593 common genes
creating a transform input..
reducing dimensionality...
clustering..


doing differential expression analysis on the clusters..
running differential expression on 25 clusters
running limmapy on the samples
you need to have R installed with the limma library installed
3.2.1
there is 0.212 overlap between the fit and transform dataset in their most variable genes
doing cPCA..
transform
regressing out the cPCA components..
> /home/jeremie/celligner/celligner/__init__.py(360)transform()
-> varsubset = np.array([1 if i in self.differential_genes_names else 0 for i in self.transform_input.columns]).astype(bool)
(Pdb) n
> /home/jeremie/celligner/celligner/__init__.py(361)transform()
-> self.corrected = mnnpy.mnn_correct(transformed_fit.values,
(Pdb) self.corrected = mnnpy.mnn_correct(transformed_fit.values,                                       transformed_transform.values,                                       var_index=list(transformed_fit.columns),                                       **self.mnn_kwargs)
Performing cosine normalization...
Starting MNN correct

(Pdb) transformed_fit.shape
(12236, 29593)
(Pdb) transformed_transform.shape
(1411, 29593)
(Pdb) mnnpy.mnn_correct(transformed_fit.values,                                       transformed_transform.values,)
*** TypeError: object of type 'NoneType' has no len()
(Pdb) mnnpy.mnn_correct(transformed_fit.values,                                       transformed_transform.values, var_index=list(range(len(transformed_fit.columns))))
Performing cosine normalization...
Starting MNN correct iteration. Reference batch: 0
Step 1 of 1: processing batch 1
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
MNN correction complete. Gathering output...
(array([[ 1.2714214e-02, -6.5224590e-03, -8.4216779e-05, ...,
         5.9781168e-03,  8.2681319e-03, -7.6961129e-05],
       [ 1.0388911e-03,  3.7117195e-03,  2.1752852e-04, ...,
         5.6056455e-03,  5.2036545e-03, -1.9303916e-05],
       [ 2.7462456e-03,  3.7217591e-04,  4.2671614e-05, ...,
   

/home/jeremie/miniconda3/lib/python3.7/site-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


*** ValueError: Shape of passed values is (3, 1), indices imply (13647, 29593)
(Pdb) self.corrected[0]
array([[ 1.2714214e-02, -6.5224590e-03, -8.4216779e-05, ...,
         5.9781168e-03,  8.2681319e-03, -7.6961129e-05],
       [ 1.0388911e-03,  3.7117195e-03,  2.1752852e-04, ...,
         5.6056455e-03,  5.2036545e-03, -1.9303916e-05],
       [ 2.7462456e-03,  3.7217591e-04,  4.2671614e-05, ...,
         5.4910104e-03,  2.9733414e-03, -3.8713162e-05],
       ...,
       [ 1.0222286e-02, -3.8719687e-03, -9.7824137e-05, ...,
         4.8552365e-03,  5.2895430e-03,  2.9971688e-05],
       [ 6.9125826e-03, -1.7929720e-03, -1.5856794e-04, ...,
         6.0051903e-03,  4.0034405e-03,  3.6182028e-05],
       [ 1.1874364e-02, -3.6757223e-03, -1.8088409e-05, ...,
         5.1361262e-03,  9.6146157e-04, -2.7215938e-05]], dtype=float32)
(Pdb) self.corrected[1]
[0,    new cell  ref cell  original batch
0         0      6736               0]
(Pdb) self.corrected, _, _  = self.corrected
(Pdb) self.

BdbQuit: 

In [14]:
my_alligner.save('temp/demo/')

In [15]:
! ls -alh temp/demo

total 4.6G
drwxr-xr-x 2 jeremie jeremie 4.0K Sep 17 20:18 .
drwxr-xr-x 4 jeremie jeremie 4.0K Sep 17 20:18 ..
-rw-r--r-- 1 jeremie jeremie 4.6G Sep 17 20:19 model.pkl


In [ ]:
my_alligner.load('temp/demo/')

In [27]:
my_alligner.plot()